In [ ]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor

from IPython.display import Audio
import os

import torchaudio

from torchaudio.transforms import Resample
from torchaudio.functional import resample
import torch


In [2]:
read_path = "/data/a.varlamov/asvspoof/ASVspoof2021_DF_eval/flac"

output_path = "/data/a.varlamov/asvspoof/DF_flacs_encodec"

In [3]:
os.listdir(read_path)

['DF_E_2000011.flac',
 'DF_E_2000013.flac',
 'DF_E_2000024.flac',
 'DF_E_2000026.flac',
 'DF_E_2000027.flac',
 'DF_E_2000028.flac',
 'DF_E_2000031.flac',
 'DF_E_2000032.flac',
 'DF_E_2000040.flac',
 'DF_E_2000042.flac',
 'DF_E_2000044.flac',
 'DF_E_2000048.flac',
 'DF_E_2000049.flac',
 'DF_E_2000053.flac',
 'DF_E_2000055.flac',
 'DF_E_2000058.flac',
 'DF_E_2000072.flac',
 'DF_E_2000075.flac',
 'DF_E_2000079.flac',
 'DF_E_2000080.flac',
 'DF_E_2000091.flac',
 'DF_E_2000096.flac',
 'DF_E_2000100.flac',
 'DF_E_2000102.flac',
 'DF_E_2000106.flac',
 'DF_E_2000115.flac',
 'DF_E_2000117.flac',
 'DF_E_2000123.flac',
 'DF_E_2000125.flac',
 'DF_E_2000126.flac',
 'DF_E_2000127.flac',
 'DF_E_2000128.flac',
 'DF_E_2000132.flac',
 'DF_E_2000137.flac',
 'DF_E_2000142.flac',
 'DF_E_2000143.flac',
 'DF_E_2000147.flac',
 'DF_E_2000151.flac',
 'DF_E_2000154.flac',
 'DF_E_2000160.flac',
 'DF_E_2000162.flac',
 'DF_E_2000165.flac',
 'DF_E_2000177.flac',
 'DF_E_2000179.flac',
 'DF_E_2000180.flac',
 'DF_E_200

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [5]:
model = EncodecModel.from_pretrained("facebook/encodec_24khz").to("cuda")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

/home/aleksandr.varlamov/miniconda3/envs/encodec/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [60]:
path = "/data/a.varlamov/LJSpeech-1.1/wavs/LJ001-0011.wav"

In [61]:
# Example usage
waveform, sample_rate = torchaudio.load(path)

waveform = resample(waveform, sample_rate, processor.sampling_rate).flatten()

In [62]:
batch = torch.cat(
    [waveform.unsqueeze(0)] * 32
).unsqueeze(1)

In [63]:
batch.shape

torch.Size([32, 1, 108283])

In [69]:
waveform.unsqueeze(0)

tensor([[[-0.0007, -0.0007, -0.0008,  ...,  0.0008,  0.0007,  0.0007]]])

In [64]:
Audio(waveform.unsqueeze(0), rate=processor.sampling_rate)

In [78]:
from tqdm import tqdm

In [109]:
def augment_with_encodec(waveform, sample_rate):
    '''
    
    '''
    waveform = resample(waveform, sample_rate, 24000).flatten()
    
    audio_values = model(waveform.to("cuda").unsqueeze(0).unsqueeze(1)).audio_values.flatten().cpu().detach()

    aug_waveform = resample(audio_values, processor.sampling_rate, sample_rate)
    
    return aug_waveform.unsqueeze(0)

In [ ]:
for i, path in tqdm(enumerate(os.listdir(read_path))):
    wave, sr = torchaudio.load(f"{read_path}/{path}")
    wave_aug = augment_with_encodec(wave, sr)
    # print(path)
    torchaudio.save(f"{output_path}/{path}", wave_aug, sample_rate=sr)

    
    if i > 10:
        break

6it [00:00, 25.31it/s]

16000
16000
16000
16000
16000
16000


9it [00:00, 24.67it/s]

16000
16000
16000
16000
16000


11it [00:00, 22.48it/s]

16000


In [100]:
Audio(wave_aug, rate=22050)

In [ ]:
torchaudio.save

In [90]:
path

'DF_E_2000031.flac'

In [111]:
import numpy as np

In [112]:
np.random.uniform(0, 1)

0.019143503347404223